In [ ]:
import sys
from pathlib import Path
src_path = str(Path().resolve().parent.parent)
if src_path != sys.path[0]:
    sys.path.insert(0, src_path)

In [5]:
import os
import json
from src.convex_mcp.server import initialize_mcp, run_convex_function, get_function_spec, get_graph_data
# from src.convex_mcp.actions import create_entities, create_relations
from src.schemas import BriefEntity, BriefRelation
from src.convex_mcp.examples import ENTITY_1,  ENTITY_2, RELATION_3

In [6]:
deployment_info = await initialize_mcp()
if not deployment_info:
    raise Exception("No deployment info found")

Initializing MCP with project dir: /Users/tanujvasudeva/Documents/GitHub/YouTwo/backend


In [7]:
function_spec = await get_function_spec(deployment_info)

In [11]:
# VALID_FUNCTIONS
for func in function_spec:
    print(func.get("identifier"))
    # if func.get("visibility", {}).get("kind") != "internal":
    #     print(func)
        # print(func.get("identifier"), func.get("visibility", {}).get("kind"))

entities.js:addObservations
entities.js:createEntities
entities.js:deleteEntities
entities.js:deleteObservations
entities.js:getBriefEntities
relations.js:createRelations
relations.js:deleteRelations
knowledge.js:readGraph


In [26]:
def get_http_url():
    if deployment_info['url']:
        return deployment_info['url']
    convex_url = os.getenv("CONVEX_URL")
    if not convex_url:
        raise ValueError("CONVEX_URL environment variable not set")
    return convex_url

graph_from_http = await get_graph_data({"deploymentSelector": None, "url": get_http_url()})

In [13]:
entities: list[BriefEntity] = [
    {
        "name": "Bacon",
        "entityType": "food"
    },
    {
        "name": "Eggs",
        "entityType": "food"
    },
    {
        "name": "Tashi",
        "entityType": "person"
    }
]
await run_convex_function(deployment_info['deploymentSelector'], "entities:createEntities", {"entities": entities})

CallToolResult(meta=None, content=[TextContent(type='text', text='{"result":[{"id":"j970cg7x2hqmxcmq4an7svf1c97hw8q2","name":"Bacon","success":true},{"id":"j9788n7m6gsdp42fzxzefedw2x7hxa94","name":"Eggs","success":true},{"id":"j970058y834wsar9y3xvkkws257hw9ph","name":"Tashi","success":true}],"logLines":[]}', annotations=None)], isError=False)

In [18]:
graph = await run_convex_function(deployment_info['deploymentSelector'], "knowledge:readGraph", {})

In [20]:
res_1 = json.loads(graph.content[0].text)["result"]

In [27]:
res_1 == graph_from_http

True

In [30]:
graph_from_http["entities"][0]

{'entityType': 'person',
 'id': 'j972py8g9baxmz4s71t47b5db57hvby2',
 'name': 'Tan',
 'observations': ['Tan often craves cheese.'],
 'updatedAt': 1749893054437}

In [31]:
res_1["relations"][0]

{'from': 'Kiren',
 'fromEntityId': 'j970eypejv9ge0eq39p6j7wm3h7hv8f8',
 'id': 'jx72g68p96s503z9jx51p87kv57hvmtd',
 'relationType': 'loves',
 'to': 'Tan',
 'toEntityId': 'j972py8g9baxmz4s71t47b5db57hvby2'}